In [1]:
# add_lch_to_clusters.py

import sqlite3
import numpy as np
import os

# Define the database path
DATABASE_PATH = os.path.join("/Users/greyson/Projects/custom_gallery/gallery/prisma", 'image_analysis.db')

def connect_db():
    conn = sqlite3.connect(DATABASE_PATH)
    conn.execute('PRAGMA foreign_keys = ON;')  # Enable foreign key support
    return conn

def add_columns_to_clusters(conn):
    cursor = conn.cursor()
    # Add 'c' (chroma) column
    cursor.execute('''
    ALTER TABLE Clusters
    ADD COLUMN c REAL
    ''')
    # Add 'h' (hue) column
    cursor.execute('''
    ALTER TABLE Clusters
    ADD COLUMN h REAL
    ''')
    conn.commit()
    print("Added 'c' and 'h' columns to Clusters table.")

def calculate_and_update_lch(conn):
    cursor = conn.cursor()
    # Fetch all cluster IDs and their LAB values
    cursor.execute('''
    SELECT clustering_id, cluster_index, l, a, b_channel
    FROM Clusters
    ''')
    clusters = cursor.fetchall()

    print(f"Calculating LCH values for {len(clusters)} clusters...")

    for cluster in clusters:
        clustering_id, cluster_index, l_val, a_val, b_val = cluster
        # Compute c (chroma)
        c_val = np.sqrt(a_val**2 + b_val**2)
        # Compute h (hue angle in degrees)
        h_rad = np.arctan2(b_val, a_val)  # Returns angle in radians
        h_deg = np.degrees(h_rad)
        # Ensure h_deg is between 0 and 360 degrees
        if h_deg < 0:
            h_deg += 360

        # Update the cluster with the calculated c and h values
        cursor.execute('''
        UPDATE Clusters
        SET c = ?, h = ?
        WHERE clustering_id = ? AND cluster_index = ?
        ''', (c_val, h_deg, clustering_id, cluster_index))

    conn.commit()
    print("LCH values calculated and updated successfully.")

def main():
    conn = connect_db()
    cursor = conn.cursor()

    # Check if 'c' and 'h' columns already exist
    cursor.execute("PRAGMA table_info(Clusters)")
    columns = [info[1] for info in cursor.fetchall()]
    if 'c' not in columns or 'h' not in columns:
        add_columns_to_clusters(conn)
    else:
        print("'c' and 'h' columns already exist in Clusters table.")

    calculate_and_update_lch(conn)
    conn.close()

if __name__ == "__main__":
    main()


Added 'c' and 'h' columns to Clusters table.
Calculating LCH values for 61968 clusters...
LCH values calculated and updated successfully.
